## Deep Learning Classifier 

### Melinda Xiao-Devins

Implement deep learning neural network classifier


In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
import os

Using TensorFlow backend.


In [2]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
#from tensorflow.python.layers.core import Dense
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))
# Check for a GPU
if not tf.test.gpu_device_name():
 warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
 print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


In [3]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [4]:
# load dataset
#df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")
df = pd.read_csv('./data/ny_hmda_2015_normalize.csv', low_memory=False, header=0, delimiter=",")
  
#print(dataframe.loc[:,:])
num_rows = df.shape[0]
num_col = df.shape[1]
print ("Total number of records: {}".format(num_rows))
print ("Toatl numver of features: {}".format(num_col))

X = np.array(df.drop(['action_taken'],1)) 
Y = np.array(df['action_taken'])

#Split into train and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

Total number of records: 376516
Toatl numver of features: 61


In [5]:
# Create a fully connected network with 4 layers
model = Sequential()

#input layer, it has 60 neurons, it must have right number of inputs, which is the number of features
model.add(Dense(60, input_dim=num_col-1, kernel_initializer='normal', activation='relu'))

# hideen layer has 32 neurons
model.add(Dense(32, kernel_initializer='normal', activation='relu'))

# hideen layer has 16 neurons
model.add(Dense(16, kernel_initializer='normal', activation='relu'))

# hideen layer has 8 neurons
model.add(Dense(8, kernel_initializer='normal', activation='relu'))

# output layer has 1 neuron to predict
# Use sigmoid for output layer activation function to ensure network output is bw. 0 and 1
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [6]:
#Compile model

# loss function: logarithmic loss, which is binary_crossentropy for binary classification
# use 'adam' optimizer for gradient descent algorithm 
# collect accuracy during training
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# Train the model
# epochs: a fixed number of iterations through the dataset
# batch size: the number of instances that are evaluated before a weight update in the network is performed 
model.fit(X_train, Y_train, epochs = 1000, batch_size=100)

Epoch 1/1000
301212/301212 [==============================] - 13s 44us/step - loss: 0.3474 - acc: 0.8202
Epoch 2/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3314 - acc: 0.8314
Epoch 3/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3296 - acc: 0.8324
Epoch 4/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3288 - acc: 0.8332
Epoch 5/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3280 - acc: 0.8331
Epoch 6/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3273 - acc: 0.8339
Epoch 7/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3272 - acc: 0.8337
Epoch 8/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3267 - acc: 0.8345
Epoch 9/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3264 - acc: 0.8346
Epoch 10/1000
301212/301212 [==========================

301212/301212 [==============================] - 12s 41us/step - loss: 0.3184 - acc: 0.8411
Epoch 79/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3182 - acc: 0.8413
Epoch 80/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3183 - acc: 0.8415
Epoch 81/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3182 - acc: 0.8410
Epoch 82/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3181 - acc: 0.8414
Epoch 83/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3181 - acc: 0.8412
Epoch 84/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3180 - acc: 0.8413
Epoch 85/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3180 - acc: 0.8416
Epoch 86/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3180 - acc: 0.8411
Epoch 87/1000
301212/301212 [==============================]

301212/301212 [==============================] - 12s 41us/step - loss: 0.3150 - acc: 0.8431
Epoch 155/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3151 - acc: 0.8433
Epoch 156/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3148 - acc: 0.8428
Epoch 157/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3149 - acc: 0.8432
Epoch 158/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3148 - acc: 0.8434
Epoch 159/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3149 - acc: 0.8435
Epoch 160/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3148 - acc: 0.8431
Epoch 161/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3147 - acc: 0.8436
Epoch 162/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3148 - acc: 0.8433
Epoch 163/1000
301212/301212 [======================

301212/301212 [==============================] - 12s 40us/step - loss: 0.3125 - acc: 0.8449
Epoch 231/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3124 - acc: 0.8452
Epoch 232/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3124 - acc: 0.8446
Epoch 233/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3123 - acc: 0.8453
Epoch 234/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3123 - acc: 0.8448
Epoch 235/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3124 - acc: 0.8447
Epoch 236/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3122 - acc: 0.8450
Epoch 237/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3122 - acc: 0.8446
Epoch 238/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3120 - acc: 0.8450
Epoch 239/1000
301212/301212 [======================

301212/301212 [==============================] - 12s 40us/step - loss: 0.3104 - acc: 0.8460
Epoch 307/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3103 - acc: 0.8461
Epoch 308/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3104 - acc: 0.8464
Epoch 309/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3102 - acc: 0.8464
Epoch 310/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3102 - acc: 0.8462
Epoch 311/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3105 - acc: 0.8461
Epoch 312/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3104 - acc: 0.8459
Epoch 313/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3101 - acc: 0.8460
Epoch 314/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3102 - acc: 0.8459
Epoch 315/1000
301212/301212 [======================

...

301212/301212 [==============================] - 12s 41us/step - loss: 0.3065 - acc: 0.8483
Epoch 687/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3064 - acc: 0.8484
Epoch 688/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3063 - acc: 0.8490
Epoch 689/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3063 - acc: 0.8488
Epoch 690/1000
301212/301212 [==============================] - 12s 39us/step - loss: 0.3065 - acc: 0.8489
Epoch 691/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3066 - acc: 0.8488
Epoch 692/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3065 - acc: 0.8489
Epoch 693/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3065 - acc: 0.8486
Epoch 694/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3064 - acc: 0.8487
Epoch 695/1000
301212/301212 [======================

301212/301212 [==============================] - 12s 40us/step - loss: 0.3061 - acc: 0.8488
Epoch 763/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3059 - acc: 0.8496
Epoch 764/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3063 - acc: 0.8487
Epoch 765/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3061 - acc: 0.8490
Epoch 766/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3061 - acc: 0.8489
Epoch 767/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3060 - acc: 0.8486
Epoch 768/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3059 - acc: 0.8490
Epoch 769/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3060 - acc: 0.8485
Epoch 770/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3061 - acc: 0.8490
Epoch 771/1000
301212/301212 [======================

301212/301212 [==============================] - 12s 41us/step - loss: 0.3057 - acc: 0.8490
Epoch 839/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3057 - acc: 0.8491
Epoch 840/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3055 - acc: 0.8489
Epoch 841/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3057 - acc: 0.8492
Epoch 842/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3057 - acc: 0.8494
Epoch 843/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3058 - acc: 0.8489
Epoch 844/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3057 - acc: 0.8491
Epoch 845/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3055 - acc: 0.8495
Epoch 846/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3055 - acc: 0.8490
Epoch 847/1000
301212/301212 [======================

301212/301212 [==============================] - 12s 40us/step - loss: 0.3051 - acc: 0.8496
Epoch 915/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3052 - acc: 0.8493
Epoch 916/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3054 - acc: 0.8496
Epoch 917/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3052 - acc: 0.8493
Epoch 918/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3051 - acc: 0.8498
Epoch 919/1000
301212/301212 [==============================] - 12s 39us/step - loss: 0.3053 - acc: 0.8495
Epoch 920/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3054 - acc: 0.8490
Epoch 921/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3051 - acc: 0.8499
Epoch 922/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3051 - acc: 0.8492
Epoch 923/1000
301212/301212 [======================

301212/301212 [==============================] - 12s 40us/step - loss: 0.3052 - acc: 0.8497
Epoch 991/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3048 - acc: 0.8498
Epoch 992/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3048 - acc: 0.8497
Epoch 993/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3049 - acc: 0.8494
Epoch 994/1000
301212/301212 [==============================] - 13s 42us/step - loss: 0.3047 - acc: 0.8496
Epoch 995/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3048 - acc: 0.8496
Epoch 996/1000
301212/301212 [==============================] - 12s 41us/step - loss: 0.3048 - acc: 0.8497
Epoch 997/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3047 - acc: 0.8499
Epoch 998/1000
301212/301212 [==============================] - 12s 40us/step - loss: 0.3049 - acc: 0.8492
Epoch 999/1000
301212/301212 [======================

In [8]:
# evaluate the model
scores = model.evaluate(X_test, Y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

75304/75304 [==============================] - 3s 38us/step

acc: 84.13%


In [9]:
# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])


the prediction bw. 0 and 1
[[ 0.59589666]
 [ 1.        ]
 [ 0.75607461]
 ..., 
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]
the rounded prediction
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [10]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(Y_test, [round(x[0]) for x in model.predict(X_test)],average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))


precision=0.8209453983821843, recall=0.8115709377175834, fscore=0.815908710022752, support=None


## Save Trained Model
Save the trained model to disk

In [11]:
 
# serialize model to JSON
model_json = model.to_json()
with open("models/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("models/model.h5")
print("Saved model to disk")

Saved model to disk


## Load Saved Model
Load the saved model, and used it. It saves training time

In [12]:
# load json and create model
json_file = open('models/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("models/model.h5")
print("Loaded model from disk")
 
print("Use loaded model to predict")
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# calculate predictions
predictions = model.predict(X)
print("the prediction bw. 0 and 1")
print(predictions)

# round predictions
rounded = [round(x[0]) for x in predictions]
print("the rounded prediction")
print(rounded[0:10])



Loaded model from disk
Use loaded model to predict
the prediction bw. 0 and 1
[[ 0.59589666]
 [ 1.        ]
 [ 0.75607461]
 ..., 
 [ 1.        ]
 [ 1.        ]
 [ 1.        ]]
the rounded prediction
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [13]:
score = loaded_model.evaluate(X_test, Y_test, verbose=0)
print("From saved model,  %s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))



From saved model,  acc: 84.13%


In [14]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(Y_test, [round(x[0]) for x in model.predict(X_test)],average="macro")
print("precision={}, recall={}, fscore={}, support={}".format(precision, recall, fscore, support))


precision=0.8209453983821843, recall=0.8115709377175834, fscore=0.815908710022752, support=None
